In [1]:
import numpy as np
import math
import sys, glob
import soundfile as sf # read audio
import sounddevice as sd
import random
import threading
import librosa # resample function
from scipy import signal # fst convolution function
from IPython.display import Audio # Audio listening in notebook
import os

In [2]:
# 현재위치 확인
os.getcwd()

'C:\\Users\\Bae JaeHyeong'

In [ ]:
# 위치 이동
os.chdir('..')

In [11]:
# sound source와 hrtf 불러오기
source_dir = '.jupyter/Sound Sample/*.wav'
hrtf_dir_LISTEN = 'Desktop/HRTFsets/IRC_1021_C/*.wav'

_SOURCES = glob.glob(source_dir)
_LISTEN = glob.glob(hrtf_dir_LISTEN)
_LISTEN

['Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T000_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T015_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T030_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T045_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T060_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T075_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T090_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T105_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T120_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T135_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T150_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T165_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T180_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T195_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_C_R0195_T210_P000.wav',
 'Desktop/HRTFsets/IRC_1021_C\\IRC_1021_

In [ ]:
# stereo audio -> mono audio, 실행시킬 필요 없음
def convert_mono(idx):
    [sig, fs_s] = sf.read(_SOURCES[idx])
    if sig.shape[1]>1:
        sig_mono = np.mean(sig,axis=1)
    else:
        sig_mono = sig
    sf.write("_mono.wav", sig_mono, fs_s)
    return

In [ ]:
# stereo audio -> mono audio, 실행시킬 필요 없음
convert_mono(1)

In [12]:
# 변환기 및 합성기
def convolve_HRTF(time = 1, max_dis = 10, half_cam_angle = 43.5, xpixel = 950):
    
    totarr = []
    min_dis = max_dis
    
    # arr[0] = type, arr[1] = d, arr[2] = x, arr[3] = y
    arr = list(map(float, input("Please input [type, depth, x, y]\n").split()))
    while arr[0]:
        newarr = [int(arr[0])]
        newarr.append(math.sqrt((2*max_dis*math.tan(half_cam_angle/180)/xpixel*abs(arr[2]-xpixel/2))**2+arr[1]**2))
        newarr.append(-math.degrees(np.arctan(2*max_dis*math.tan(half_cam_angle/180)/xpixel*(arr[2]-xpixel/2)/arr[1])))
        if(arr[2] > xpixel/2):
            newarr[2] = 360 + newarr[2]
        if(newarr[1] < min_dis):
            min_dis = newarr[1]
        print(newarr)
        totarr.append(newarr)
        arr = list(map(float, input().split()))

    L = 0
    R = 0

    # type = arr[0], distance = arr[1], degree = arr[2]
    for arr in totarr:
        idx = round(arr[2]/15) % 24
        print(_LISTEN[idx])

        # Load HRTF sets
        HRIR, fs_H = librosa.load(_LISTEN[idx], sr=48000, mono=False)

        # Load source to spatialize
        [src, fs_s] = librosa.load(_SOURCES[arr[0]-1], mono=True, sr=48000)

        # Convolve -> Frequency domain is faster
        s_L = signal.fftconvolve(src,HRIR[0,:]) # spatialized source L
        s_R = signal.fftconvolve(src,HRIR[1,:]) # spatialized source R

        if(len(s_L) != 48883):
            s_L = np.pad(s_L,(0,(48883-len(s_L))),'constant')
            s_R = np.pad(s_R,(0,(48883-len(s_R))),'constant')
        
        # Add L signals and R signals, create stereo file
        L += s_L * (max_dis - arr[1]) / max_dis
        R += s_R * (max_dis - arr[1]) / max_dis

    HRTF_Mix = np.vstack([L,R]).transpose()

    # Scale the amplitude to peak at 1 (normalize)
    HRTF_Mix = HRTF_Mix/np.max(np.abs(HRTF_Mix)) * (max_dis - min_dis) / max_dis
    sf.write(f'{time}.wav',HRTF_Mix,48000)

    # Audio
    sd.play(HRTF_Mix, 48000, device=None)

    return

In [13]:
# 변환기 및 합성기(자동화)
def convolve_HRTF_auto(time = 1, max_dis = 10, half_cam_angle = 43.5, xpixel = 950):
    
    totarr, infoarr = [], []
    min_dis = max_dis
    
    # arr[0] = type, arr[1] = d, arr[2] = x, arr[3] = y
    
    f = open("Desktop/info.txt", 'r')
    while 1:
        line = f.readline()
        if not line:
            break
        infoarr.append(list(map(int, line.split())))
    for arr in infoarr:
        newarr = [arr[0]]
        newarr.append(math.sqrt((2*max_dis*math.tan(half_cam_angle/180)/xpixel*abs(arr[2]-xpixel/2))**2+arr[1]**2))
        newarr.append(-math.degrees(np.arctan(2*max_dis*math.tan(half_cam_angle/180)/xpixel*(arr[2]-xpixel/2)/arr[1])))
        if(arr[2] > xpixel/2):
            newarr[2] = 360 + newarr[2]
        if(newarr[1] < min_dis):
            min_dis = newarr[1]
        print(newarr)
        totarr.append(newarr)

    L = 0
    R = 0

    # type = arr[0], distance = arr[1], degree = arr[2]
    for arr in totarr:
        idx = round(arr[2]/15) % 24
        print(_LISTEN[idx])

        # Load HRTF sets
        HRIR, fs_H = librosa.load(_LISTEN[idx], sr=48000, mono=False)

        # Load source to spatialize
        [src, fs_s] = librosa.load(_SOURCES[arr[0]-1], mono=True, sr=48000)

        # Convolve -> Frequency domain is faster
        s_L = signal.fftconvolve(src,HRIR[0,:]) # spatialized source L
        s_R = signal.fftconvolve(src,HRIR[1,:]) # spatialized source R

        if(len(s_L) != 48883):
            s_L = np.pad(s_L,(0,(48883-len(s_L))),'constant')
            s_R = np.pad(s_R,(0,(48883-len(s_R))),'constant')
        
        # Add L signals and R signals, create stereo file
        L += s_L * (max_dis - arr[1]) / max_dis
        R += s_R * (max_dis - arr[1]) / max_dis

    HRTF_Mix = np.vstack([L,R]).transpose()

    # Scale the amplitude to peak at 1 (normalize)
    HRTF_Mix = HRTF_Mix/np.max(np.abs(HRTF_Mix)) * (max_dis - min_dis) / max_dis
    sf.write(f'{time}.wav',HRTF_Mix,48000)

    # Audio
    sd.play(HRTF_Mix, 48000, device=None)

    return

In [ ]:
convolve_HRTF()

In [14]:
convolve_HRTF_auto()

[1, 1.1388026588676001, 331.4158382220707]
[2, 2.4569011854794915, 35.50808646951002]
Desktop/HRTFsets/IRC_1021_C\IRC_1021_C_R0195_T330_P000.wav
Desktop/HRTFsets/IRC_1021_C\IRC_1021_C_R0195_T030_P000.wav


In [5]:
totarr = [[1,3,30]]

def test_HRTF_1(max_dis = 10):
    # arr[0] = type, arr[1] = d, arr[2] = x, arr[3] = y
    L = 0
    R = 0

    # type = arr[0], distance = arr[1], degree = arr[2]
    for arr in totarr:
        idx = round(arr[2]/15)

        # Load HRTF sets
        HRIR, fs_H = librosa.load(_LISTEN[idx], sr=48000, mono=False)

        # Load source to spatialize
        [src, fs_s] = librosa.load(_SOURCES[arr[0]-1], mono=True, sr=48000)

        # Convolve -> Frequency domain is faster
        s_L = signal.fftconvolve(src,HRIR[0,:]) # spatialized source L
        s_R = signal.fftconvolve(src,HRIR[1,:]) # spatialized source R

        if(len(s_L) != 48883):
            s_L = np.pad(s_L,(0,(48883-len(s_L))),'constant')
            s_R = np.pad(s_R,(0,(48883-len(s_R))),'constant')
        
        # Add L signals and R signals, create stereo file
        L += s_L * (max_dis - arr[1] / max_dis)
        R += s_R * (max_dis - arr[1] / max_dis)

    HRTF_Mix = np.vstack([L,R]).transpose()

    # Scale the amplitude to peak at 1 (normalize)
    HRTF_Mix = HRTF_Mix/np.max(np.abs(HRTF_Mix)) * (max_dis - min_dis) / max_dis
    sd.play(HRTF_Mix, 48000, device=None)
    input('')
    return

test_HRTF_1()
totarr = [[1,3,30],[1,1,345]]
test_HRTF_1()
test_HRTF_1()
test_HRTF_1()
test_HRTF_1()
totarr = [[2,5,315],[3,1,345]]
test_HRTF_1()

NameError: name 'min_dis' is not defined

In [6]:
def test_HRTF(max_dis = 10):
    angle_list = [0, 15, 30, 45, 315, 330, 345]
    totarr = []
    
    list_sound = [1, 2, 3]
    type_list = [random.choice(list_sound) for i in range(random.choice(list_sound))]
    type_list.sort()
    for typ in type_list:
        totarr.append(list([typ]))
    
    L, R, min_dis = 0, 0, 10

    # type = arr[0], distance = arr[1], degree = arr[2]
    for i in range(len(type_list)):
        dis = random.randrange(11)
        totarr[i].append(dis)
        if dis < min_dis:
            min_dis = dis
        totarr[i].append(random.choice(angle_list))
        idx = totarr[i][2]//15

        # Load HRTF sets
        HRIR, fs_H = librosa.load(_LISTEN[idx], sr=48000, mono=False)

        # Load source to spatialize
        [src, fs_s] = librosa.load(_SOURCES[totarr[i][0]-1], mono=True, sr=48000)

        # Convolve -> Frequency domain is faster
        s_L = signal.fftconvolve(src,HRIR[0,:]) # spatialized source L
        s_R = signal.fftconvolve(src,HRIR[1,:]) # spatialized source R

        if(len(s_L) != 48883):
            s_L = np.pad(s_L,(0,(48883-len(s_L))),'constant')
            s_R = np.pad(s_R,(0,(48883-len(s_R))),'constant')
        
        # Add L signals and R signals, create stereo file
        L += s_L * (max_dis - totarr[i][1] / max_dis)
        R += s_R * (max_dis - totarr[i][1] / max_dis)

    HRTF_Mix = np.vstack([L,R]).transpose()
    
    # Scale the amplitude to peak at 1 (normalize)
    HRTF_Mix = HRTF_Mix/np.max(np.abs(HRTF_Mix)) * (max_dis - min_dis) / max_dis
    
    def play_sound(sound):
        sd.play(sound, 48000, device=None)

    play_sound(HRTF_Mix)
    num = int(input("Predict the number of obstacle. If you want to play sound one more time, input 0\n"))
    while(num == 0):
        play_sound(HRTF_Mix)
        num = int(input("Predict the number of obstacle. If you want to play sound one more time, input 0\n"))
    while(num != len(type_list)):
        num = int(input("Wrong answer, Please listen careful and predict the number of obstacles"))
        play_sound(HRTF_Mix)
    print("Right Answer")
    for i in range(num):
        toggle = 0
        arr = input(f'Predict the [type, dis, angle] of obstacle{i+1}.\ntype: 1-car, 2-people, 3-wall / dis: 0~10 / angle: 0~45, 315~345\n')
        for j in range(len(type_list)):
            if(totarr[j] == arr):
                toggle = 1
                totarr.pop(j)
                break
        if(toggle):
            print("Correct!")
        else:
            print("Wrong")
    print(totarr)
    return


In [ ]:
test_HRTF()